# Exercises 1 - MNIST Optical Character Recognition
### Course: Convolutional Neural Networks with Applications in Medical Image Analysis
Office hours: Mondays 13.15--15.00 (Tommy), Tuesdays 13.15--16.00 (Minh), Thursdays 08.15--12.00 (Attila)

Below is an example notebook for a simple Keras pipeline. The dataset is MNIST (http://yann.lecun.com/exdb/mnist/), where each image is of a handwritten digit of 0-9.

In [ ]:
# Import needed packages

from __future__ import print_function
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import backend as K
from tensorflow.python.client import device_lib
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [ ]:
# See if GPU is available to use

print(device_lib.list_local_devices())

In [ ]:
# Load in MNIST data
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Input image dimensions
img_rows, img_cols = np.shape(x_train)[1:] # This will keep original size. Change if you want to resize
batch_size = 128
num_classes = 10
epochs = 15

In [ ]:
# Formatting stuff
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

In [ ]:
# Split dataset into training and testing dataset
x_train, x_validate, y_train, y_validate = train_test_split(x_train, y_train, test_size=0.1, random_state=101)
print(x_train.shape[0], 'train samples')
print(x_validate.shape[0], 'validation samples')

In [ ]:
# Convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_validate = keras.utils.to_categorical(y_validate, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

In [ ]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])

model.summary()

In [ ]:
history = model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=2,
          validation_data=(x_validate, y_validate))

In [ ]:
# Plotting the losses for the training and validation data
plt.figure()
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['loss', 'val_loss'])
plt.show()

In [ ]:
# Plotting the recognition accuracies for the training and validation data
plt.figure()
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.legend(['acc', 'val_acc'])
plt.show()

In [ ]:
# Evaluate test error
score = model.evaluate(x_test, y_test, verbose=0)

In [ ]:
print(f"Test loss    : {score[0]:.3f}")
print(f"Test accuracy: {score[1]:.3f}")

In [ ]:
# See what we actually did:
plt.figure()
plt.imshow(x_test[0, :, :, 0], cmap='gray')
plt.show()

guesses = model.predict(x_test)

# Chance of image being of each character 0-9
print(guesses[0])

# Where is the maximum?
print()
print('The maximum is at: ' + str(np.argmax(guesses[0])))

In [ ]:
# Plotting some results

n = 4
plt.figure(figsize=(13, 13))
guesses = model.predict(x_test[0:n * n, :, :, :])
for i in range(n * n):
    plt.subplot(n, n, i + 1)
    plt.imshow(x_test[i, :, :, 0], cmap='gray')
    plt.axis('off')
    plt.title(str(np.argmax(guesses[i])) + " (" + str(int(guesses[i, np.argmax(guesses[i])] * 100)) + "%)")